In [152]:

import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import pooch
from scipy.sparse import csr_matrix
from scipy.io import mmwrite
import re
import anndata
import scanpy.external as sce
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import math
import requests
from mygene import MyGeneInfo
import re

In [153]:
df_log = pd.read_csv("E:\Research_umich\Professor_He\IU_hackthon\edge_generator\markers_ttest_scanpy_log_mean_value.csv")

<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:1: SyntaxWarning: invalid escape sequence '\R'
C:\Users\ruope\AppData\Local\Temp\ipykernel_20056\2384769598.py:1: SyntaxWarning: invalid escape sequence '\R'
  df_log = pd.read_csv("E:\Research_umich\Professor_He\IU_hackthon\edge_generator\markers_ttest_scanpy_log_mean_value.csv")


In [154]:
df_log

,group,names,scores,logfoldchanges,pvals,pvals_adj,means
0,B cell,ENSG00000144218,40.114967,5.587823,8.842046e-229,3.467048e-227,2.009148
1,B cell,ENSG00000166501,40.027252,6.607472,4.329809e-228,1.686698e-226,1.764662
2,B cell,ENSG00000153064,39.824738,6.699023,1.609573e-226,6.216175e-225,1.973676
3,B cell,ENSG00000075884,37.953392,6.240255,5.293878e-212,1.873114e-210,1.747170
4,B cell,ENSG00000111913,35.823612,5.416226,1.981515e-195,6.370322e-194,1.622804
...,...,...,...,...,...,...,...
455,neural cell,ENSG00000133121,14.887564,4.165942,1.470213e-31,5.154221e-31,2.115563
456,neural cell,ENSG00000071991,14.757838,11.175801,3.256385e-31,1.138054e-30,1.784673
457,neural cell,ENSG00000196376,14.755793,7.936396,3.296976e-31,1.152127e-30,1.799449
458,neural cell,ENSG00000154654,14.107642,7.895089,1.764037e-29,6.058791e-29,1.680517


In [155]:
def ensg_to_gene_and_iri(ensg_id: str, species="human"):

    ensg_core = re.sub(r"\.\d+$", "", ensg_id)

    mg = MyGeneInfo()
    hits = mg.querymany([ensg_core],
                        scopes="ensembl.gene",
                        fields="symbol,name,hgnc,HGNC",
                        species=species, as_dataframe=False)
    hit = next((h for h in hits if not h.get("notfound")), None)

    symbol = name = None
    hgnc_id = None
    if hit:
        symbol = hit.get("symbol")
        name = hit.get("name")
        hgnc_id = hit.get("HGNC", hit.get("hgnc"))

    
    if symbol is None or name is None:
        url = f"https://rest.ensembl.org/lookup/id/{ensg_core}"
        r = requests.get(url, headers={"Content-Type": "application/json"})
        if r.ok:
            j = r.json()
            symbol = symbol or j.get("display_name")
            name = name or j.get("description")

    ensembl_iri = f"https://identifiers.org/ensembl:{ensg_core}"
    hgnc_iri = f"https://identifiers.org/hgnc/{int(hgnc_id)}" if hgnc_id else None
    return {
        "ensembl_id": ensg_core,
        "symbol": symbol,
        "gene_name": name,
        "hgnc_id": hgnc_id,
        "ensembl_iri": ensembl_iri,
        "hgnc_iri": hgnc_iri,
    }

info = ensg_to_gene_and_iri("ENSG00000153064")
print(info)

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


{'ensembl_id': 'ENSG00000153064', 'symbol': 'BANK1', 'gene_name': 'B cell scaffold protein with ankyrin repeats 1', 'hgnc_id': '18233', 'ensembl_iri': 'https://identifiers.org/ensembl:ENSG00000153064', 'hgnc_iri': 'https://identifiers.org/hgnc/18233'}


In [156]:
df_log

,group,names,scores,logfoldchanges,pvals,pvals_adj,means
0,B cell,ENSG00000144218,40.114967,5.587823,8.842046e-229,3.467048e-227,2.009148
1,B cell,ENSG00000166501,40.027252,6.607472,4.329809e-228,1.686698e-226,1.764662
2,B cell,ENSG00000153064,39.824738,6.699023,1.609573e-226,6.216175e-225,1.973676
3,B cell,ENSG00000075884,37.953392,6.240255,5.293878e-212,1.873114e-210,1.747170
4,B cell,ENSG00000111913,35.823612,5.416226,1.981515e-195,6.370322e-194,1.622804
...,...,...,...,...,...,...,...
455,neural cell,ENSG00000133121,14.887564,4.165942,1.470213e-31,5.154221e-31,2.115563
456,neural cell,ENSG00000071991,14.757838,11.175801,3.256385e-31,1.138054e-30,1.784673
457,neural cell,ENSG00000196376,14.755793,7.936396,3.296976e-31,1.152127e-30,1.799449
458,neural cell,ENSG00000154654,14.107642,7.895089,1.764037e-29,6.058791e-29,1.680517


In [157]:
df_log.iloc[41] 

group             Collecting duct principal cell
names                            ENSG00000150201
scores                                 220.09157
logfoldchanges                          3.331373
pvals                                        0.0
pvals_adj                                    0.0
means                                   1.191442
Name: 41, dtype: object

In [158]:
def ensembl_to_symbols_and_iris(ids, species="human"):
    ids = pd.Series(ids, dtype="string")
    core = ids.str.replace(r"\.\d+$", "", regex=True)  # strip version suffix
    uniq = core.dropna().unique().tolist()

    mg = MyGeneInfo()
    hits = mg.querymany(
        uniq,
        scopes="ensembl.gene",
        fields="symbol,name,hgnc,HGNC",
        species=species,
        as_dataframe=False,
    )

    rows = []
    for h in hits:
        ensg = h.get("query")
        if h.get("notfound"):
            rows.append({"ensembl_id": ensg, "symbol": None, "hgnc_id": None})
        else:
            rows.append({
                "ensembl_id": ensg,
                "symbol": h.get("symbol"),
                "hgnc_id": h.get("HGNC", h.get("hgnc")),
            })

    map_df = pd.DataFrame(rows).drop_duplicates("ensembl_id")

    out = pd.DataFrame({"input_id": ids, "ensembl_id": core})
    out = out.merge(map_df, on="ensembl_id", how="left")

    out["ensembl_iri"] = "https://identifiers.org/ensembl/" + out["ensembl_id"].astype(str)
    out["hgnc_iri"] = out["hgnc_id"].apply(
        lambda x: f"https://identifiers.org/hgnc/{int(x)}" if pd.notna(x) else None
    )
    return out

In [159]:
ids = ["ENSG00000251562"]
df = ensembl_to_symbols_and_iris(df_log['names'])
print(df)

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
1 input query terms found no hit:	['ENSG00000287523']


            input_id       ensembl_id    symbol hgnc_id  \
0    ENSG00000144218  ENSG00000144218      AFF3    6473   
1    ENSG00000166501  ENSG00000166501     PRKCB    9395   
2    ENSG00000153064  ENSG00000153064     BANK1   18233   
3    ENSG00000075884  ENSG00000075884  ARHGAP15   21030   
4    ENSG00000111913  ENSG00000111913    RIPOR2   13872   
..               ...              ...       ...     ...   
455  ENSG00000133121  ENSG00000133121   STARD13   19164   
456  ENSG00000071991  ENSG00000071991     CDH19    1758   
457  ENSG00000196376  ENSG00000196376   SLC35F1   21483   
458  ENSG00000154654  ENSG00000154654     NCAM2    7657   
459  ENSG00000230490  ENSG00000230490      None    None   

                                         ensembl_iri  \
0    https://identifiers.org/ensembl/ENSG00000144218   
1    https://identifiers.org/ensembl/ENSG00000166501   
2    https://identifiers.org/ensembl/ENSG00000153064   
3    https://identifiers.org/ensembl/ENSG00000075884   
4    https:

In [160]:
lookup = df.drop_duplicates("ensembl_id").set_index("ensembl_id")["hgnc_iri"]
df_new = df_log.assign(iri=df_log["names"].map(lookup))
df_new.rename(columns={"iri": "gene_iri"}, inplace=True)

In [161]:
df

,input_id,ensembl_id,symbol,hgnc_id,ensembl_iri,hgnc_iri
0,ENSG00000144218,ENSG00000144218,AFF3,6473,https://identifiers.org/ensembl/ENSG00000144218,https://identifiers.org/hgnc/6473
1,ENSG00000166501,ENSG00000166501,PRKCB,9395,https://identifiers.org/ensembl/ENSG00000166501,https://identifiers.org/hgnc/9395
2,ENSG00000153064,ENSG00000153064,BANK1,18233,https://identifiers.org/ensembl/ENSG00000153064,https://identifiers.org/hgnc/18233
3,ENSG00000075884,ENSG00000075884,ARHGAP15,21030,https://identifiers.org/ensembl/ENSG00000075884,https://identifiers.org/hgnc/21030
4,ENSG00000111913,ENSG00000111913,RIPOR2,13872,https://identifiers.org/ensembl/ENSG00000111913,https://identifiers.org/hgnc/13872
...,...,...,...,...,...,...
455,ENSG00000133121,ENSG00000133121,STARD13,19164,https://identifiers.org/ensembl/ENSG00000133121,https://identifiers.org/hgnc/19164
456,ENSG00000071991,ENSG00000071991,CDH19,1758,https://identifiers.org/ensembl/ENSG00000071991,https://identifiers.org/hgnc/1758
457,ENSG00000196376,ENSG00000196376,SLC35F1,21483,https://identifiers.org/ensembl/ENSG00000196376,https://identifiers.org/hgnc/21483
458,ENSG00000154654,ENSG00000154654,NCAM2,7657,https://identifiers.org/ensembl/ENSG00000154654,https://identifiers.org/hgnc/7657


In [162]:
for idx, row in df.iterrows():
    print(idx, row)


df_node = pd.DataFrame(columns=["iri", "label","type", "source"])
for idx, row in df.iterrows():
    iri="http://identifiers.org/hgnc/"+str(row['hgnc_id'])
    df_node = pd.concat([df_node, pd.DataFrame([{
    "iri": iri,
    "label": row['symbol'],
    "type": "http://purl.bioontology.org/ontology/HGNC/gene",
    "source": "https://github.com/hubmapconsortium/hra-hubmap-kpmp-integration"
    }])], ignore_index=True)

0 input_id                                       ENSG00000144218
ensembl_id                                     ENSG00000144218
symbol                                                    AFF3
hgnc_id                                                   6473
ensembl_iri    https://identifiers.org/ensembl/ENSG00000144218
hgnc_iri                     https://identifiers.org/hgnc/6473
Name: 0, dtype: object
1 input_id                                       ENSG00000166501
ensembl_id                                     ENSG00000166501
symbol                                                   PRKCB
hgnc_id                                                   9395
ensembl_iri    https://identifiers.org/ensembl/ENSG00000166501
hgnc_iri                     https://identifiers.org/hgnc/9395
Name: 1, dtype: object
2 input_id                                       ENSG00000153064
ensembl_id                                     ENSG00000153064
symbol                                                   BANK1
hgn

In [163]:
df_node

,iri,label,type,source
0,http://identifiers.org/hgnc/6473,AFF3,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
1,http://identifiers.org/hgnc/9395,PRKCB,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
2,http://identifiers.org/hgnc/18233,BANK1,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
3,http://identifiers.org/hgnc/21030,ARHGAP15,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
4,http://identifiers.org/hgnc/13872,RIPOR2,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
...,...,...,...,...
455,http://identifiers.org/hgnc/19164,STARD13,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
456,http://identifiers.org/hgnc/1758,CDH19,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
457,http://identifiers.org/hgnc/21483,SLC35F1,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
458,http://identifiers.org/hgnc/7657,NCAM2,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...


In [164]:
def get_cell_ontology_info_list(cell_types: list[str]) -> pd.DataFrame:
    """
    Query OLS (Ontology Lookup Service) for a list of cell types.
    Returns a DataFrame with columns: input, iri, label, obo_id, short_form.
    """
    records = []

    for ct in cell_types:
        url = "https://www.ebi.ac.uk/ols/api/search"
        params = {
            "q": ct,
            "ontology": "cl",  
            "exact": "false"
        }
        r = requests.get(url, params=params)

        if r.status_code == 200:
            data = r.json()
            if data['response']['numFound'] > 0:
                hit = data['response']['docs'][0]
                records.append({
                    "input": ct,
                    "iri": hit.get("iri"),
                    "label": hit.get("label"),
                    "obo_id": hit.get("obo_id"),
                    "short_form": hit.get("short_form"),
                    "source": "http://purl.obolibrary.org/obo/" + hit.get("short_form")
                })
            else:
                records.append({"input": ct, "iri": None, "label": None, 
                                "obo_id": None, "short_form": None, "source": None})
        else:
            records.append({"input": ct, "iri": None, "label": None, 
                            "obo_id": None, "short_form": None, "source": None})

    return pd.DataFrame(records)


cell_types = df_log['group'].unique()
df_cell_types = get_cell_ontology_info_list(cell_types)

In [165]:
df_cell_types

,input,iri,label,obo_id,short_form,source
0,B cell,http://purl.obolibrary.org/obo/CL_0000236,B cell,CL:0000236,CL_0000236,http://purl.obolibrary.org/obo/CL_0000236
1,Collecting duct intercalated cell,http://purl.obolibrary.org/obo/CL_1001432,kidney collecting duct intercalated cell,CL:1001432,CL_1001432,http://purl.obolibrary.org/obo/CL_1001432
2,Collecting duct principal cell,http://purl.obolibrary.org/obo/CL_1001431,kidney collecting duct principal cell,CL:1001431,CL_1001431,http://purl.obolibrary.org/obo/CL_1001431
3,Connecting tubule epithelial cell,http://purl.obolibrary.org/obo/CL_1000768,kidney connecting tubule epithelial cell,CL:1000768,CL_1000768,http://purl.obolibrary.org/obo/CL_1000768
4,Dendritic cell,http://purl.obolibrary.org/obo/CL_0000451,dendritic cell,CL:0000451,CL_0000451,http://purl.obolibrary.org/obo/CL_0000451
5,Distal convoluted tubule epithelial cell,http://purl.obolibrary.org/obo/CL_1000849,kidney distal convoluted tubule epithelial cell,CL:1000849,CL_1000849,http://purl.obolibrary.org/obo/CL_1000849
6,Endothelial cell,http://purl.obolibrary.org/obo/CL_0000115,endothelial cell,CL:0000115,CL_0000115,http://purl.obolibrary.org/obo/CL_0000115
7,Kidney interstitial alternatively activated ma...,http://purl.obolibrary.org/obo/CL_1000695,kidney interstitial alternatively activated ma...,CL:1000695,CL_1000695,http://purl.obolibrary.org/obo/CL_1000695
8,Kidney interstitial fibroblast,http://purl.obolibrary.org/obo/CL_1000692,kidney interstitial fibroblast,CL:1000692,CL_1000692,http://purl.obolibrary.org/obo/CL_1000692
9,Loop of Henle epithelial cell,http://purl.obolibrary.org/obo/CL_1000909,kidney loop of Henle epithelial cell,CL:1000909,CL_1000909,http://purl.obolibrary.org/obo/CL_1000909


In [166]:
lookup_n = df_cell_types.drop_duplicates("input").set_index("input")["source"]
df_new = df_new.assign(iri=df_new["group"].map(lookup_n))
df_new.rename(columns={"iri": "cell_iri"}, inplace=True)

In [167]:
df_new

,group,names,scores,logfoldchanges,pvals,pvals_adj,means,gene_iri,cell_iri
0,B cell,ENSG00000144218,40.114967,5.587823,8.842046e-229,3.467048e-227,2.009148,https://identifiers.org/hgnc/6473,http://purl.obolibrary.org/obo/CL_0000236
1,B cell,ENSG00000166501,40.027252,6.607472,4.329809e-228,1.686698e-226,1.764662,https://identifiers.org/hgnc/9395,http://purl.obolibrary.org/obo/CL_0000236
2,B cell,ENSG00000153064,39.824738,6.699023,1.609573e-226,6.216175e-225,1.973676,https://identifiers.org/hgnc/18233,http://purl.obolibrary.org/obo/CL_0000236
3,B cell,ENSG00000075884,37.953392,6.240255,5.293878e-212,1.873114e-210,1.747170,https://identifiers.org/hgnc/21030,http://purl.obolibrary.org/obo/CL_0000236
4,B cell,ENSG00000111913,35.823612,5.416226,1.981515e-195,6.370322e-194,1.622804,https://identifiers.org/hgnc/13872,http://purl.obolibrary.org/obo/CL_0000236
...,...,...,...,...,...,...,...,...,...
455,neural cell,ENSG00000133121,14.887564,4.165942,1.470213e-31,5.154221e-31,2.115563,https://identifiers.org/hgnc/19164,http://purl.obolibrary.org/obo/CL_0002319
456,neural cell,ENSG00000071991,14.757838,11.175801,3.256385e-31,1.138054e-30,1.784673,https://identifiers.org/hgnc/1758,http://purl.obolibrary.org/obo/CL_0002319
457,neural cell,ENSG00000196376,14.755793,7.936396,3.296976e-31,1.152127e-30,1.799449,https://identifiers.org/hgnc/21483,http://purl.obolibrary.org/obo/CL_0002319
458,neural cell,ENSG00000154654,14.107642,7.895089,1.764037e-29,6.058791e-29,1.680517,https://identifiers.org/hgnc/7657,http://purl.obolibrary.org/obo/CL_0002319


In [168]:
df_node

,iri,label,type,source
0,http://identifiers.org/hgnc/6473,AFF3,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
1,http://identifiers.org/hgnc/9395,PRKCB,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
2,http://identifiers.org/hgnc/18233,BANK1,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
3,http://identifiers.org/hgnc/21030,ARHGAP15,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
4,http://identifiers.org/hgnc/13872,RIPOR2,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
...,...,...,...,...
455,http://identifiers.org/hgnc/19164,STARD13,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
456,http://identifiers.org/hgnc/1758,CDH19,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
457,http://identifiers.org/hgnc/21483,SLC35F1,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
458,http://identifiers.org/hgnc/7657,NCAM2,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...


In [169]:
for idx, row in df_cell_types.iterrows():
    print(idx, row)


df_node_for_cell_types = pd.DataFrame(columns=["iri", "label","type", "source"])
for idx, row in df_cell_types.iterrows():
    df_node = pd.concat([df_node, pd.DataFrame([{
    "iri": row["iri"],
    "label": row["input"],
    "type": "http://purl.obolibrary.org/obo/CL_0000000",
    "source": "https://github.com/hubmapconsortium/hra-hubmap-kpmp-integration"
    }])], ignore_index=True)

0 input                                            B cell
iri           http://purl.obolibrary.org/obo/CL_0000236
label                                            B cell
obo_id                                       CL:0000236
short_form                                   CL_0000236
source        http://purl.obolibrary.org/obo/CL_0000236
Name: 0, dtype: object
1 input                 Collecting duct intercalated cell
iri           http://purl.obolibrary.org/obo/CL_1001432
label          kidney collecting duct intercalated cell
obo_id                                       CL:1001432
short_form                                   CL_1001432
source        http://purl.obolibrary.org/obo/CL_1001432
Name: 1, dtype: object
2 input                    Collecting duct principal cell
iri           http://purl.obolibrary.org/obo/CL_1001431
label             kidney collecting duct principal cell
obo_id                                       CL:1001431
short_form                                   CL_1001

In [170]:
df_node

,iri,label,type,source
0,http://identifiers.org/hgnc/6473,AFF3,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
1,http://identifiers.org/hgnc/9395,PRKCB,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
2,http://identifiers.org/hgnc/18233,BANK1,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
3,http://identifiers.org/hgnc/21030,ARHGAP15,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
4,http://identifiers.org/hgnc/13872,RIPOR2,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
...,...,...,...,...
478,http://purl.obolibrary.org/obo/CL_0002306,Proximal tubule epithelial cell,http://purl.obolibrary.org/obo/CL_0000000,https://github.com/hubmapconsortium/hra-hubmap...
479,http://purl.obolibrary.org/obo/CL_1001318,Renal interstitial pericyte,http://purl.obolibrary.org/obo/CL_0000000,https://github.com/hubmapconsortium/hra-hubmap...
480,http://purl.obolibrary.org/obo/CL_0000084,T cell,http://purl.obolibrary.org/obo/CL_0000000,https://github.com/hubmapconsortium/hra-hubmap...
481,http://purl.obolibrary.org/obo/CL_1001106,kidney loop of Henle thick ascending limb epit...,http://purl.obolibrary.org/obo/CL_0000000,https://github.com/hubmapconsortium/hra-hubmap...


In [171]:
df_node = df_node[df_node["iri"] != "http://identifiers.org/hgnc/None	"]


In [172]:

if "Unnamed: 0" in df_node.columns:
    df_node = df_node.drop("Unnamed: 0", axis=1)

In [173]:
df_node

,iri,label,type,source
0,http://identifiers.org/hgnc/6473,AFF3,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
1,http://identifiers.org/hgnc/9395,PRKCB,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
2,http://identifiers.org/hgnc/18233,BANK1,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
3,http://identifiers.org/hgnc/21030,ARHGAP15,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
4,http://identifiers.org/hgnc/13872,RIPOR2,http://purl.bioontology.org/ontology/HGNC/gene,https://github.com/hubmapconsortium/hra-hubmap...
...,...,...,...,...
478,http://purl.obolibrary.org/obo/CL_0002306,Proximal tubule epithelial cell,http://purl.obolibrary.org/obo/CL_0000000,https://github.com/hubmapconsortium/hra-hubmap...
479,http://purl.obolibrary.org/obo/CL_1001318,Renal interstitial pericyte,http://purl.obolibrary.org/obo/CL_0000000,https://github.com/hubmapconsortium/hra-hubmap...
480,http://purl.obolibrary.org/obo/CL_0000084,T cell,http://purl.obolibrary.org/obo/CL_0000000,https://github.com/hubmapconsortium/hra-hubmap...
481,http://purl.obolibrary.org/obo/CL_1001106,kidney loop of Henle thick ascending limb epit...,http://purl.obolibrary.org/obo/CL_0000000,https://github.com/hubmapconsortium/hra-hubmap...


In [174]:
df_node.to_csv("hra-kpmp-hubmap-nodes.csv", index=False)

In [175]:
df_log

,group,names,scores,logfoldchanges,pvals,pvals_adj,means
0,B cell,ENSG00000144218,40.114967,5.587823,8.842046e-229,3.467048e-227,2.009148
1,B cell,ENSG00000166501,40.027252,6.607472,4.329809e-228,1.686698e-226,1.764662
2,B cell,ENSG00000153064,39.824738,6.699023,1.609573e-226,6.216175e-225,1.973676
3,B cell,ENSG00000075884,37.953392,6.240255,5.293878e-212,1.873114e-210,1.747170
4,B cell,ENSG00000111913,35.823612,5.416226,1.981515e-195,6.370322e-194,1.622804
...,...,...,...,...,...,...,...
455,neural cell,ENSG00000133121,14.887564,4.165942,1.470213e-31,5.154221e-31,2.115563
456,neural cell,ENSG00000071991,14.757838,11.175801,3.256385e-31,1.138054e-30,1.784673
457,neural cell,ENSG00000196376,14.755793,7.936396,3.296976e-31,1.152127e-30,1.799449
458,neural cell,ENSG00000154654,14.107642,7.895089,1.764037e-29,6.058791e-29,1.680517


In [176]:
df

,input_id,ensembl_id,symbol,hgnc_id,ensembl_iri,hgnc_iri
0,ENSG00000144218,ENSG00000144218,AFF3,6473,https://identifiers.org/ensembl/ENSG00000144218,https://identifiers.org/hgnc/6473
1,ENSG00000166501,ENSG00000166501,PRKCB,9395,https://identifiers.org/ensembl/ENSG00000166501,https://identifiers.org/hgnc/9395
2,ENSG00000153064,ENSG00000153064,BANK1,18233,https://identifiers.org/ensembl/ENSG00000153064,https://identifiers.org/hgnc/18233
3,ENSG00000075884,ENSG00000075884,ARHGAP15,21030,https://identifiers.org/ensembl/ENSG00000075884,https://identifiers.org/hgnc/21030
4,ENSG00000111913,ENSG00000111913,RIPOR2,13872,https://identifiers.org/ensembl/ENSG00000111913,https://identifiers.org/hgnc/13872
...,...,...,...,...,...,...
455,ENSG00000133121,ENSG00000133121,STARD13,19164,https://identifiers.org/ensembl/ENSG00000133121,https://identifiers.org/hgnc/19164
456,ENSG00000071991,ENSG00000071991,CDH19,1758,https://identifiers.org/ensembl/ENSG00000071991,https://identifiers.org/hgnc/1758
457,ENSG00000196376,ENSG00000196376,SLC35F1,21483,https://identifiers.org/ensembl/ENSG00000196376,https://identifiers.org/hgnc/21483
458,ENSG00000154654,ENSG00000154654,NCAM2,7657,https://identifiers.org/ensembl/ENSG00000154654,https://identifiers.org/hgnc/7657


In [177]:
def get_gene_id_by_using_df(df: pd.DataFrame, gene_name: str):
    """
    Given a gene name, find the matching row(s) in df['gene_name']
    and return the corresponding df['gene_id'].
    """
    matches = df[df["input_id"] == gene_name]

    if matches.empty:
        return None   
    return matches["hgnc_iri"].tolist()
print(get_gene_id_by_using_df(df, "ENSG00000153064"))

['https://identifiers.org/hgnc/18233']


In [178]:
def get_cell_id_by_using_df(df: pd.DataFrame, gene_name: str):
    """
    Given a gene name, find the matching row(s) in df['gene_name']
    and return the corresponding df['gene_id'].
    """
    matches = df_cell_types[df_cell_types["input"] == gene_name]

    if matches.empty:
        return None   
    return matches["iri"].tolist()
print(get_cell_id_by_using_df(df_cell_types, "T cell"))

['http://purl.obolibrary.org/obo/CL_0000084']


In [179]:
df_log

,group,names,scores,logfoldchanges,pvals,pvals_adj,means
0,B cell,ENSG00000144218,40.114967,5.587823,8.842046e-229,3.467048e-227,2.009148
1,B cell,ENSG00000166501,40.027252,6.607472,4.329809e-228,1.686698e-226,1.764662
2,B cell,ENSG00000153064,39.824738,6.699023,1.609573e-226,6.216175e-225,1.973676
3,B cell,ENSG00000075884,37.953392,6.240255,5.293878e-212,1.873114e-210,1.747170
4,B cell,ENSG00000111913,35.823612,5.416226,1.981515e-195,6.370322e-194,1.622804
...,...,...,...,...,...,...,...
455,neural cell,ENSG00000133121,14.887564,4.165942,1.470213e-31,5.154221e-31,2.115563
456,neural cell,ENSG00000071991,14.757838,11.175801,3.256385e-31,1.138054e-30,1.784673
457,neural cell,ENSG00000196376,14.755793,7.936396,3.296976e-31,1.152127e-30,1.799449
458,neural cell,ENSG00000154654,14.107642,7.895089,1.764037e-29,6.058791e-29,1.680517


In [180]:
for idx, row in df_node.iterrows():
    print(idx, row)

df_edge = pd.DataFrame(columns=["subject", "predicate","object", "source"])
for idx, row in df_log.iterrows():
    df_edge = pd.concat([df_edge, pd.DataFrame([{
    "subject": get_gene_id_by_using_df(df,row['names'])[0],
    "predicate": "http://purl.org/ccf/ccf_characterizes",
    "object": get_cell_id_by_using_df(df_cell_types,row['group'])[0],
    "source": "https://github.com/hubmapconsortium/hra-hubmap-kpmp-integration"
    }])], ignore_index=True)

0 iri                        http://identifiers.org/hgnc/6473
label                                                  AFF3
type         http://purl.bioontology.org/ontology/HGNC/gene
source    https://github.com/hubmapconsortium/hra-hubmap...
Name: 0, dtype: object
1 iri                        http://identifiers.org/hgnc/9395
label                                                 PRKCB
type         http://purl.bioontology.org/ontology/HGNC/gene
source    https://github.com/hubmapconsortium/hra-hubmap...
Name: 1, dtype: object
2 iri                       http://identifiers.org/hgnc/18233
label                                                 BANK1
type         http://purl.bioontology.org/ontology/HGNC/gene
source    https://github.com/hubmapconsortium/hra-hubmap...
Name: 2, dtype: object
3 iri                       http://identifiers.org/hgnc/21030
label                                              ARHGAP15
type         http://purl.bioontology.org/ontology/HGNC/gene
source    https://githu

In [181]:
df_edge

,subject,predicate,object,source
0,https://identifiers.org/hgnc/6473,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...
1,https://identifiers.org/hgnc/9395,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...
2,https://identifiers.org/hgnc/18233,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...
3,https://identifiers.org/hgnc/21030,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...
4,https://identifiers.org/hgnc/13872,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...
...,...,...,...,...
455,https://identifiers.org/hgnc/19164,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...
456,https://identifiers.org/hgnc/1758,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...
457,https://identifiers.org/hgnc/21483,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...
458,https://identifiers.org/hgnc/7657,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...


In [182]:
map_tbl = (
    df_new[["gene_iri", "cell_iri", "means"]]
      .dropna(subset=["gene_iri", "cell_iri"])
      .astype({"gene_iri": str, "cell_iri": str})
      .groupby(["gene_iri", "cell_iri"], as_index=False)["means"].mean()  
      .rename(columns={"means": "gene_expr"})
)


df_edge_norm = df_edge.copy()
df_edge_norm["subject"] = df_edge_norm["subject"].astype(str).str.strip()
df_edge_norm["object"]  = df_edge_norm["object"].astype(str).str.strip()


df_edge_with_expr = (
    df_edge_norm.merge(
        map_tbl,
        left_on=["subject", "object"],
        right_on=["gene_iri", "cell_iri"],
        how="left",
        validate="many_to_one"
    )
    .drop(columns=["gene_iri", "cell_iri"])  
)


In [183]:
df_edge_with_expr = df_edge_with_expr.dropna(subset=["gene_expr"])

In [184]:
df_edge_with_expr

,subject,predicate,object,source,gene_expr
0,https://identifiers.org/hgnc/6473,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...,2.009148
1,https://identifiers.org/hgnc/9395,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...,1.764662
2,https://identifiers.org/hgnc/18233,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...,1.973676
3,https://identifiers.org/hgnc/21030,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...,1.747170
4,https://identifiers.org/hgnc/13872,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...,1.622804
...,...,...,...,...,...
454,https://identifiers.org/hgnc/9393,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...,2.065267
455,https://identifiers.org/hgnc/19164,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...,2.115563
456,https://identifiers.org/hgnc/1758,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...,1.784673
457,https://identifiers.org/hgnc/21483,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...,1.799449


In [185]:
df_edge_with_expr

,subject,predicate,object,source,gene_expr
0,https://identifiers.org/hgnc/6473,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...,2.009148
1,https://identifiers.org/hgnc/9395,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...,1.764662
2,https://identifiers.org/hgnc/18233,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...,1.973676
3,https://identifiers.org/hgnc/21030,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...,1.747170
4,https://identifiers.org/hgnc/13872,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0000236,https://github.com/hubmapconsortium/hra-hubmap...,1.622804
...,...,...,...,...,...
454,https://identifiers.org/hgnc/9393,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...,2.065267
455,https://identifiers.org/hgnc/19164,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...,2.115563
456,https://identifiers.org/hgnc/1758,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...,1.784673
457,https://identifiers.org/hgnc/21483,http://purl.org/ccf/ccf_characterizes,http://purl.obolibrary.org/obo/CL_0002319,https://github.com/hubmapconsortium/hra-hubmap...,1.799449


In [186]:
df_edge_with_expr.to_csv("hra-kpmp-hubmap-edges.csv",index=False)